In [115]:
import lz4.block
import lz4.frame
import os

def buf2int(buf):
    v = 0
    for i, b in enumerate(buf):
        v += int(b) << (8*i)
    return v
def buf2hex(buf):
    return '.'.join(['%02x' % xx for xx in buf])

class AEDAT4(object):
    def __init__(self, filename):
        self.filename = filename
        
        with open(filename, 'rb') as f:
            buf = bytearray(f.read())
            assert buf[:14] == b'#!AER-DAT4.0\r\n'
            
            size = buf2int(buf[14:18])
            self.header_unknown = buf[18:52]
            assert self.header_unknown[4:8] == b'IOHE'
            self.info = buf[18+52:size+14].decode()
            self.header_unknown2 = buf[size+14:size+14+4+8]
            
            index = size+14+4+8
            
            self.blocks = []
            while True:
                assert buf[index:index+7] == b'\x04"M\x18@@\xc0'
                info = lz4.frame.get_frame_info(buf[index:])
                ctx = lz4.frame.create_decompression_context()
                d = b''
                while True:
                    data, size, finished = lz4.frame.decompress_chunk(ctx, buf[index:])
                    d += data
                    if finished:
                        index += size + 8
                        break
                    else:                        
                        index += size
                self.blocks.append(d)
                
                datatype = d[8:12].decode()
                print(datatype, len(d))            
            
f = AEDAT4('Drone_Cam102_test1-2020_01_30_13_30_05.aedat4')

#print(f.info)
#print(buf2int(f.header_unknown[:4]), f.header_unknown[4:8].decode(), buf2hex(f.header_unknown))
#print(buf2int(f.header_unknown2[:4]), len(f.header_unknown2), f.header_unknown2)
        
#print(len(f.blocks[0]))

EVTS 2432
TRIG 64
EVTS 2624
IMUS 520
TRIG 80
IMUS 520
EVTS 2976
TRIG 64
FRME 90064
IMUS 520
EVTS 2752
IMUS 520
EVTS 2704
IMUS 520
TRIG 64
EVTS 2992
IMUS 520
TRIG 80
EVTS 2560
FRME 90064
IMUS 520
TRIG 64
EVTS 3280
IMUS 520
EVTS 3392
IMUS 520
TRIG 64
EVTS 2880
IMUS 568
TRIG 80
EVTS 3248
TRIG 64
FRME 90064
IMUS 520
EVTS 3312
IMUS 520
EVTS 3104
IMUS 520
TRIG 64
EVTS 3296
IMUS 576
TRIG 80
EVTS 3104
TRIG 64
EVTS 2896
FRME 90064
IMUS 520
IMUS 560
EVTS 3072
IMUS 520
TRIG 64
EVTS 2944
IMUS 520
TRIG 80
EVTS 2976
TRIG 64
FRME 90064
IMUS 520
EVTS 2992
IMUS 520
TRIG 64
EVTS 2928
IMUS 520
EVTS 2976
IMUS 520
TRIG 80
EVTS 2720
TRIG 64
EVTS 2848
FRME 90064
IMUS 520
IMUS 520
EVTS 2912
IMUS 520
TRIG 64
EVTS 2736
TRIG 80
EVTS 2464
FRME 90064
IMUS 560
TRIG 64
IMUS 520
EVTS 2608
IMUS 520
TRIG 64
EVTS 2448
IMUS 520
EVTS 2688
IMUS 520
TRIG 80
EVTS 2352
TRIG 64
EVTS 2480
FRME 90064
IMUS 520
IMUS 520
EVTS 2352
IMUS 520
TRIG 64
EVTS 2640
IMUS 560
TRIG 80
EVTS 2336
FRME 90064
IMUS 520
TRIG 64
EVTS 2512
IMUS 520
E

FRME 90064
IMUS 520
IMUS 536
EVTS 1536
IMUS 560
TRIG 64
EVTS 1568
IMUS 520
TRIG 80
EVTS 1568
FRME 90064
IMUS 520
TRIG 64
EVTS 1392
IMUS 520
EVTS 1472
IMUS 520
TRIG 64
EVTS 1600
IMUS 520
TRIG 80
EVTS 1424
TRIG 64
FRME 90064
IMUS 520
EVTS 1456
IMUS 520
TRIG 64
EVTS 1552
IMUS 520
EVTS 1488
IMUS 520
TRIG 80
EVTS 1488
TRIG 64
FRME 90064
IMUS 520
EVTS 1456
IMUS 520
EVTS 1456
IMUS 520
TRIG 64
EVTS 1616
IMUS 520
TRIG 80
EVTS 1632
FRME 90064
IMUS 520
TRIG 64
EVTS 1616
IMUS 520
TRIG 64
EVTS 1696
IMUS 520
EVTS 1648
IMUS 520
TRIG 80
EVTS 1536
TRIG 64
EVTS 1632
FRME 90064
IMUS 520
IMUS 520
EVTS 1632
IMUS 520
TRIG 64
EVTS 1568
IMUS 520
TRIG 80
EVTS 1584
FRME 90064
IMUS 520
TRIG 64
EVTS 1504
IMUS 520
EVTS 1568
IMUS 520
TRIG 64
EVTS 1632
IMUS 520
TRIG 80
EVTS 1792
TRIG 64
EVTS 1536
FRME 90064
IMUS 520
IMUS 520
EVTS 1536
IMUS 520
TRIG 64
EVTS 1552
IMUS 520
TRIG 80
EVTS 1472
FRME 90064
IMUS 520
TRIG 64
EVTS 1488
IMUS 536
EVTS 1536
IMUS 520
TRIG 64
EVTS 1808
IMUS 520
TRIG 80
EVTS 1584
TRIG 64
FRME 90064


AssertionError: 

In [71]:
len(f.blocks)

11

In [54]:
f.blocks[6]

b'\x9c\x0b\x00\x00\x10\x00\x00\x00EVTS\x00\x00\x06\x00\x08\x00\x04\x00\x06\x00\x00\x00\x04\x00\x00\x00\xb8\x00\x00\x00~\xb6^\xa5_\x9d\x05\x00\x8d\x00q\x00\x00\x00\x00\x00\xa0\xb6^\xa5_\x9d\x05\x00\xc5\x00e\x00\x00\x00\x00\x00\xb9\xb6^\xa5_\x9d\x05\x00\x90\x00q\x00\x00\x00\x00\x00\xcb\xb6^\xa5_\x9d\x05\x00\x98\x00l\x00\x01\x00\x00\x00\x1c\xb7^\xa5_\x9d\x05\x00\x96\x00p\x00\x00\x00\x00\x00,\xb7^\xa5_\x9d\x05\x00\xc5\x00e\x00\x00\x00\x00\x00-\xb7^\xa5_\x9d\x05\x00\x8e\x00p\x00\x00\x00\x00\x00c\xb7^\xa5_\x9d\x05\x00\x87\x00q\x00\x00\x00\x00\x00\xb9\xb7^\xa5_\x9d\x05\x00\xc5\x00e\x00\x00\x00\x00\x00\xf5\xb7^\xa5_\x9d\x05\x00\x90\x00\x02\x01\x01\x00\x00\x00\r\xb8^\xa5_\x9d\x05\x00\x8c\x00p\x00\x00\x00\x00\x00\x1e\xb8^\xa5_\x9d\x05\x00\x93\x00q\x00\x00\x00\x00\x00E\xb8^\xa5_\x9d\x05\x00\xc5\x00e\x00\x00\x00\x00\x00\xb3\xb8^\xa5_\x9d\x05\x00\x99\x00l\x00\x01\x00\x00\x00\xd1\xb8^\xa5_\x9d\x05\x00\xc5\x00e\x00\x00\x00\x00\x00\xd5\xb8^\xa5_\x9d\x05\x00\x88\x00q\x00\x00\x00\x00\x00\x1c\xb9^\xa5_\x

In [59]:
def display(d):
    for i in range(0, len(d), 8):
        print(' '.join('%02x'%xx for xx in d[i:i+8]))
        
display(f.blocks[6])        



9c 0b 00 00 10 00 00 00
45 56 54 53 00 00 06 00
08 00 04 00 06 00 00 00
04 00 00 00 b8 00 00 00
7e b6 5e a5 5f 9d 05 00
8d 00 71 00 00 00 00 00
a0 b6 5e a5 5f 9d 05 00
c5 00 65 00 00 00 00 00
b9 b6 5e a5 5f 9d 05 00
90 00 71 00 00 00 00 00
cb b6 5e a5 5f 9d 05 00
98 00 6c 00 01 00 00 00
1c b7 5e a5 5f 9d 05 00
96 00 70 00 00 00 00 00
2c b7 5e a5 5f 9d 05 00
c5 00 65 00 00 00 00 00
2d b7 5e a5 5f 9d 05 00
8e 00 70 00 00 00 00 00
63 b7 5e a5 5f 9d 05 00
87 00 71 00 00 00 00 00
b9 b7 5e a5 5f 9d 05 00
c5 00 65 00 00 00 00 00
f5 b7 5e a5 5f 9d 05 00
90 00 02 01 01 00 00 00
0d b8 5e a5 5f 9d 05 00
8c 00 70 00 00 00 00 00
1e b8 5e a5 5f 9d 05 00
93 00 71 00 00 00 00 00
45 b8 5e a5 5f 9d 05 00
c5 00 65 00 00 00 00 00
b3 b8 5e a5 5f 9d 05 00
99 00 6c 00 01 00 00 00
d1 b8 5e a5 5f 9d 05 00
c5 00 65 00 00 00 00 00
d5 b8 5e a5 5f 9d 05 00
88 00 71 00 00 00 00 00
1c b9 5e a5 5f 9d 05 00
9c 00 6c 00 01 00 00 00
29 b9 5e a5 5f 9d 05 00
86 00 71 00 00 00 00 00
5e b9 5e a5 5f 9d 05 00
c5 00 65 00 00 0